In [19]:
import tensorflow as tf
from tensorflow.keras.applications import Xception
from sklearn.model_selection import train_test_split
import numpy as np


In [20]:
# Load the data using image_dataset_from_directory()
data_dir = "D:\Z\Data_Set\Pencil shading (2)"
image_size = (224, 224)
batch_size = 20

In [21]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,  # 80% training, 20% validation
    subset='training',
    seed=42,
    image_size=image_size,
    batch_size=batch_size
)


Found 3062 files belonging to 5 classes.
Using 2450 files for training.


In [22]:
val_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    seed=42,
    image_size=image_size,
    batch_size=batch_size
)

Found 3062 files belonging to 5 classes.
Using 612 files for validation.


In [23]:
# Configure the data pipeline
AUTOTUNE = tf.data.AUTOTUNE
train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
val_data = val_data.cache().prefetch(buffer_size=AUTOTUNE)

In [24]:
# Use ImageDataGenerator for data augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [25]:
# Apply data augmentation to the training dataset
train_data = train_data.map(lambda x, y: (data_augmentation(x, training=True), y))


In [26]:
# Split the data into training and testing sets using sklearn
X = []
y = []
for images, labels in train_data:
    X.append(images.numpy())
    y.append(labels.numpy())
X = np.concatenate(X)
y = np.concatenate(y)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [28]:
# Preprocess the image data
X_train = X_train / 255.0
X_test = X_test / 255.0

In [29]:
# Load the Xception model
base_model = Xception(
    include_top=False,
    weights='imagenet',
    input_shape=(image_size[0], image_size[1], 3)
)

In [30]:
# Freeze the base model layers
base_model.trainable = False

In [31]:
# Add custom classification head
inputs = tf.keras.Input(shape=(image_size[0], image_size[1], 3))
x = tf.keras.applications.xception.preprocess_input(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(5, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

In [32]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [33]:
# Train the model
epochs = 25
batch_size = 20
model.fit(
    X_train, y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test)
)

Epoch 1/25
98/98 [==============================] - 29s 179ms/step - loss: 1.1875 - accuracy: 0.5974 - val_loss: 1.1789 - val_accuracy: 0.5694
Epoch 2/25
98/98 [==============================] - 16s 159ms/step - loss: 1.1123 - accuracy: 0.6020 - val_loss: 1.1133 - val_accuracy: 0.5694
Epoch 3/25
98/98 [==============================] - 18s 181ms/step - loss: 1.0549 - accuracy: 0.6163 - val_loss: 1.0562 - val_accuracy: 0.6000
Epoch 4/25
98/98 [==============================] - 16s 159ms/step - loss: 0.9947 - accuracy: 0.6577 - val_loss: 1.0122 - val_accuracy: 0.6653
Epoch 5/25
98/98 [==============================] - 16s 159ms/step - loss: 0.9549 - accuracy: 0.6847 - val_loss: 0.9858 - val_accuracy: 0.6592
Epoch 6/25
98/98 [==============================] - 16s 159ms/step - loss: 0.9235 - accuracy: 0.6939 - val_loss: 0.9355 - val_accuracy: 0.6735
Epoch 7/25
98/98 [==============================] - 16s 159ms/step - loss: 0.8952 - accuracy: 0.6974 - val_loss: 0.9157 - val_accuracy: 0.6837

In [34]:
# Save the model
model.save('xception_model.h5')
